<a href="https://colab.research.google.com/github/jaeohshin/ML_with_Pytorch_Sklearn_rasbt/blob/main/ch14_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [ ]:
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([
            zero_pad, x_padded, zero_pad
        ])
    res = []
    for i in range(0, int((len(x_padded)-len(w_rot)))+1, s):
        res.append(np.sum(
            x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

In [ ]:
x = [1, 3, 2, 4, 5, 6]
w = [1, 2, 3]
print(conv1d(x, w, p=0, s=1))

[11 17 19 28]


In [ ]:
import scipy.signal

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_org = np.array(X)
    n1 = X_org.shape[0] + 2*p[0]
    n2 = X_org.shape[1] + 2*p[1]

    ## make zero matrix with padded size.
    X_padded = np.zeros(shape=(n1, n2))

    ## replace the inner part of the matrix with the original matrix
    X_padded[p[0]:p[0]+X_org.shape[0], p[1]:p[1]+X_org.shape[1]] = X_org

    res = []
    for i in range(0, (X_padded.shape[0]-W_rot.shape[0])+1, 1):
        res.append([])
        for j in range(0, (X_padded.shape[1] - W_rot.shape[1])+1, 1):

            # Select the sub-field (matrix) for convolution calculation
            X_sub = X_padded[i:i + W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

In [ ]:
X = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

W = [
    [1, 0, 3],
    [1, 2, 1],
    [0, 1, 1]
]

print('Conv2d Implementation:\n', conv2d(X, W, p=(1, 1), s = (1, 1)))

Conv2d Implementation:
 [[ 9. 26. 23.]
 [22. 53. 46.]
 [26. 41. 37.]]


파이토치를 사용하여 심층 합성곱 신경망 구현

In [5]:
import torch
import torchvision
from torchvision import transforms
image_path = './'
transform = transforms.Compose([transforms.ToTensor()])
mnist_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=True,
                                           transform=transform,
                                           download=True)

from torch.utils.data import Subset
mnist_valid_dataset = Subset(mnist_dataset, torch.arange(10000))
mnist_train_dataset = Subset(mnist_dataset, torch.arange(10000, len(mnist_dataset)))
mnist_test_dataset = torchvision.datasets.MNIST(root=image_path,
                                                train=False,
                                                transform=transform,
                                                download=False)


In [6]:
from torch.utils.data import DataLoader

batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset, batch_size, shuffle=False)

Constructing a CNN in PyTorch